### Importation des modules 

In [3]:
import pandas as pd
from sklearn.datasets import load_files
import numpy as np
import pickle
from tqdm import tqdm
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re, nltk
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from gensim.models import Doc2Vec
from yellowbrick.cluster import KElbowVisualizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fousseyni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fousseyni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Fousseyni\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Je n'ai pris que la colonne "review" de l'ensemble de données car j'essaie de créer un modèle de clustering basé sur les avis.

# 📥 Charger les données

In [ ]:
data_test=load_files('texte/test/')
data_train=load_files('texte/train')

In [ ]:
data_train.data

In [ ]:
data_conca= np.concatenate([np.array(data_train.data),np.array(data_test.data)])

In [72]:
df=pd.DataFrame(data_conca)

In [73]:
# Suppression des balises html des documents
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    return text

df[0] = df[0].apply(cleanText)


C:\Users\Fousseyni\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [74]:

df

,0
0,Full of (then) unknown actors TSF is a great b...
1,Amount of disappointment I am getting these da...
2,"The future, we are told, are what we make of i..."
3,Dan Katzir has produced a wonderful film that ...
4,If you want Scream or anything like the big-st...
...,...
99995,After watching Tipping the Velvet by Sarah wat...
99996,"The three main characters are all hopeless, an..."
99997,I haven't reviewed on IMDb before but this doc...
99998,Forget what I said about Emeril. Rachael Ray i...


# Conversion de texte en minuscules

In [ ]:
df[0] = df[0].apply(lambda x: x.lower()) 

# Suppression des mots vides

In [76]:
english_stopwords = stopwords.words("english")
df[0] = df[0].apply(lambda x: " ".join(x for x in x.split() if x not in english_stopwords))

# Suppression des mots non anglais et des mots d'une longueur inférieure à 3

In [77]:
words = set(nltk.corpus.words.words())
df[0] = df[0].apply(lambda x: " ".join(i for i in nltk.wordpunct_tokenize(x) if i in words and len(i) > 2))

In [78]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

# Transformer les mots en sa forme racine

In [79]:
lm = WordNetLemmatizer() 
df[0] = df[0].apply(lambda x: ' '.join(lm.lemmatize(i) for i in x.split()))

In [80]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Fousseyni\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Remplacer tout autre chose que les mots et les espaces par un espace.

In [81]:
df[0] = df[0].str.replace('[^\w\s]',' ')

C:\Users\Fousseyni\AppData\Local\Temp\ipykernel_15540\2361613010.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df[0] = df[0].str.replace('[^\w\s]',' ')


# Préparation des données pour la formation doc2vec

In [82]:
LabeledSentence = gensim.models.doc2vec.TaggedDocument
all_content_train = []
j=0
for em in df[0].values:
    all_content_train.append(LabeledSentence(em,[j]))
    j+=1
print('Nombre de textes traités: ', j)

Number of texts processed:  100000


# Modèle de formation doc2vec

In [83]:
d2v_model = Doc2Vec(all_content_train, vector_size = 100, window = 10, min_count = 500, workers=7, dm = 1,alpha=0.025, min_alpha=0.001)
d2v_model.train(all_content_train, total_examples=d2v_model.corpus_count, epochs=10, start_alpha=0.002, end_alpha=-0.016)

# Détermination du nombre de clusters

In [84]:
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0)
    kmeans.fit(d2v_model.dv.vectors_docs)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11),wcss)
plt.title('La méthode ELBOW')
plt.xlabel('le nombre de clusters')
plt.ylabel('WCSS')
plt.show()

C:\Users\Fousseyni\AppData\Local\Temp\ipykernel_15540\2076997197.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  kmeans.fit(d2v_model.docvecs.vectors_docs)


AttributeError: 'KeyedVectors' object has no attribute 'vectors_docs'

# Modèle de formation doc2vec

In [2]:
kmeans_model = KMeans(n_clusters=4, init='k-means++', max_iter=100, n_init=10,random_state=0) 
X = kmeans_model.fit(d2v_model.docvecs.vectors_docs)

NameError: name 'KMeans' is not defined

# Tracé des clusters

In [ ]:
labels = kmeans_model.labels_.tolist()
l = kmeans_model.fit_predict(d2v_model.docvecs.vectors_docs)
pca = PCA(n_components=2).fit(d2v_model.docvecs.vectors_docs)
datapoint = pca.transform(d2v_model.docvecs.vectors_docs)

plt.figure
label1 = ['#FFFF00', '#008000', '#0000FF', '#800080']
color = [label1[i] for i in labels]
plt.scatter(datapoint[:, 0], datapoint[:, 1], c=color)
centroids = kmeans_model.cluster_centers_
centroidpoint = pca.transform(centroids)
plt.scatter(centroidpoint[:, 0], centroidpoint[:, 1], marker='^', s=150, c='#000000')
plt.show()

# Sauvegarde du modèle et chargement

In [86]:
pickle.dump(kmeans_model, open('model_v1', 'wb'))
loaded_model = pickle.load(open('model_v1', 'rb'))

In [ ]:
df['label'] = loaded_model.labels_

# tracer la distribution

In [ ]:
df['label'].value_counts().plot(kind='bar', figsize=(15,8))

In [ ]:
# Tracer un cluster
label = df[df['label'] == 0]
print(label.shape)
text = ''
for i in label.index:
    text += ' ' + label['review'][i]

# Créez et générez une image de nuage de mots :
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)

# Affichez l'image générée :
plt.figure(figsize = (15,15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()